# **Project Title**
#### *Project Subtitle*

## Hypothesis;

Project thesis

In [ ]:
# Importing dependencies
import pandas as pd

# **Data**

## Dataset

Dataset details

In [1]:
# Reading in data
# (potentially?)

### EDA

In [1]:
# Beginning EDA

# **Ramona's Code Space**

*End Code Space*

# **Christian's Code Space**

*End Code Space*

# **Leigh's Code Space**

*End Code Space*

# **Angelica's Code Space**

In [32]:
#import required libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# # create a function to get the address of the each location and add it to a dataframe 
# def address_Addition(business,address1,new_df):
#     new_df['business_name'] = business
#     address_list = address1.split(',')
#     new_df['address'] = address_list[0]
#     new_df['city'] = address_list[1]
#     return new_df

# initiate driver
driver = webdriver.Chrome(service = ChromeService(ChromeDriverManager().install()))

# url list with business locations
#url = ['https://www.google.com/maps/place/Tim+Hortons/@43.7607366,-79.5321831,14z/data=!4m10!1m2!2m1!1stim+hortons!3m6!1s0x882b31d93eab2809:0xa9ea7bb65f9da6ec!8m2!3d43.7607366!4d-79.4992241!15sCgt0aW0gaG9ydG9ucyIDiAEBWg0iC3RpbSBob3J0b25zkgEKcmVzdGF1cmFudOABAA!16s%2Fg%2F1vyxk0xz','https://www.google.com/maps/place/Tim+Hortons/@43.7607366,-79.5321831,14z/data=!4m10!1m2!2m1!1stim+hortons!3m6!1s0x882b302d70a29891:0xc279061e4a5c71bc!8m2!3d43.756124!4d-79.5152637!15sCgt0aW0gaG9ydG9ucyIDiAEBWg0iC3RpbSBob3J0b25zkgEKcmVzdGF1cmFudOABAA!16s%2Fg%2F1td38wkb']
url = ["https://www.google.com/maps/place/McDonald's/@43.7607329,-79.5321831,14z/data=!4m10!1m2!2m1!1sMcDonald's!3m6!1s0x882b31e6d3859eb1:0xc92a9af2d1385093!8m2!3d43.7624131!4d-79.490243!15sCgpNY0RvbmFsZCdzIgOIAQFaDCIKbWNkb25hbGQnc5IBFGZhc3RfZm9vZF9yZXN0YXVyYW504AEA!16s%2Fg%2F1hc604hjv?entry=ttu"]


driver.get(url[0])
time.sleep(5)

# Find the address of the location
response = BeautifulSoup(driver.page_source, 'html.parser')
business_name = response.find('h1',class_='DUwDvf lfPIob').text
business_name



    # address = response.find('div',class_= 'rogA2c').text

"McDonald's"

In [24]:
business_name

In [33]:
#import required libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# create a function to get the address of the each location and add it to a dataframe 
def business_Overview(business,avg_rating,address1,new_df):
    new_df['business_name'] = business
    new_df['avg_rating'] = avg_rating
    address_list = address1.split(',')
    new_df['address'] = address_list[0]
    new_df['city'] = address_list[1]
    return new_df

# initiate driver
driver = webdriver.Chrome(service = ChromeService(ChromeDriverManager().install()))

# url list with business locations
#url = ['https://www.google.com/maps/place/Tim+Hortons/@43.7607366,-79.5321831,14z/data=!4m10!1m2!2m1!1stim+hortons!3m6!1s0x882b31d93eab2809:0xa9ea7bb65f9da6ec!8m2!3d43.7607366!4d-79.4992241!15sCgt0aW0gaG9ydG9ucyIDiAEBWg0iC3RpbSBob3J0b25zkgEKcmVzdGF1cmFudOABAA!16s%2Fg%2F1vyxk0xz','https://www.google.com/maps/place/Tim+Hortons/@43.7607366,-79.5321831,14z/data=!4m10!1m2!2m1!1stim+hortons!3m6!1s0x882b302d70a29891:0xc279061e4a5c71bc!8m2!3d43.756124!4d-79.5152637!15sCgt0aW0gaG9ydG9ucyIDiAEBWg0iC3RpbSBob3J0b25zkgEKcmVzdGF1cmFudOABAA!16s%2Fg%2F1td38wkb']
url = ["https://www.google.com/maps/place/McDonald's/@43.7607329,-79.5321831,14z/data=!4m10!1m2!2m1!1sMcDonald's!3m6!1s0x882b31e6d3859eb1:0xc92a9af2d1385093!8m2!3d43.7624131!4d-79.490243!15sCgpNY0RvbmFsZCdzIgOIAQFaDCIKbWNkb25hbGQnc5IBFGZhc3RfZm9vZF9yZXN0YXVyYW504AEA!16s%2Fg%2F1hc604hjv?entry=ttu"]


#create for loop to parse through the different locations in the url list above
c = 0
for i in range(0,len(url)):
    c = c+1
    driver.get(url[i])
    time.sleep(5)

    # Find the address of the location
    response = BeautifulSoup(driver.page_source, 'html.parser')
    business_name = response.find('h1',class_='DUwDvf lfPIob').text
    avg_rating = response.find('div',class_='fontDisplayLarge').text
    address = response.find('div',class_= 'rogA2c').text
    driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]').click()
    time.sleep(3)
    
    # By default, only 10 reviews can be extracted - to extract more reviews we have to scroll down the apge
    SCROLL_PAUSE_TIME = 5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    number = 0

    while True:
        number = number+1

        # Scroll down to bottom

        #old_==ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
        ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]')
        driver.execute_script('arguments[0].scrollBy(0, 5000);', ele)

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        #print(f'last height: {last_height}')

        ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]')

        new_height = driver.execute_script("return arguments[0].scrollHeight", ele)

        #print(f'new height: {new_height}')

        if number == 5:
            break

        if new_height == last_height:
            break

        #print('cont')
        last_height = new_height
    next_item = driver.find_elements('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[9]')
    time.sleep(3)

    #expand review by click on 'more' button
    for i in next_item:
        button = i.find_elements(By.TAG_NAME,'button')
        for m in button:
            if m.text == "More":
                m.click()
        time.sleep(5)

    response = BeautifulSoup(driver.page_source, 'html.parser')
    next_2 = response.find_all('div',class_ = 'jftiEf')

    #get review by passing it to a dictionary
    def get_review_summary(result_set):
        rev_dict = {
            'Review Name': [],
            'Review Text' : [],
            'Review Rating' : []}

        for result in result_set:
            review_name = result.find(class_='d4r55').text
            review_text = result.find('span',class_='wiI7pd').text
            review_rating = result.find(class_='kvMYJc')['aria-label']
            rev_dict['Review Name'].append(review_name)
            rev_dict['Review Text'].append(review_text)
            rev_dict['Review Rating'].append(review_rating)
        
         
        return(pd.DataFrame(rev_dict))

    df = get_review_summary(next_2)
    if c == 1:
        df1 = df.copy()
        final_df = business_Overview(business_name,avg_rating,address,df1)
    else:
        df2 = df.copy()
        final_df = business_Overview(business_name,avg_rating,address,df2)
        final_df = pd.concat([df1,final_df],axis = 0)
    

print(final_df)

       Review Name                                        Review Text  \
0  Tamer Arslantaş  I wanted to take a coffee.  So i came here. I ...   
1     Gabriel Seah  This outlet is below average for McDonald's.\n...   
2     meysam emily  All is good atleast in our order. My question ...   

  Review Rating business_name avg_rating        address         city  
0       5 stars    McDonald's        3.8  3929 Keele St   North York  
1        1 star    McDonald's        3.8  3929 Keele St   North York  
2       4 stars    McDonald's        3.8  3929 Keele St   North York  


In [34]:
# review df with reviews and locations
final_df


,Review Name,Review Text,Review Rating,business_name,avg_rating,address,city
0,Tamer Arslantaş,I wanted to take a coffee. So i came here. I ...,5 stars,McDonald's,3.8,3929 Keele St,North York
1,Gabriel Seah,This outlet is below average for McDonald's.\n...,1 star,McDonald's,3.8,3929 Keele St,North York
2,meysam emily,All is good atleast in our order. My question ...,4 stars,McDonald's,3.8,3929 Keele St,North York


*End Code Space*

# **Odele's Code Space**

*End Code Space*

# **Vanessa's Code Space**

*End Code Space*

# **Train Test Splitting**

# **Scaling and Encoding**

# **Modeling**

# **Application (?)**

# **Findings**

# **Citations and Licenses**

## Citations

## Licenses